# Sistema Integrado de Trading WDO

Este notebook demonstra o funcionamento completo do sistema.

In [ ]:
# Configuração inicial
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# Configura caminhos
PROJECT_ROOT = Path(os.path.dirname(os.getcwd()))  # Sobe um nível da pasta notebooks
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Carrega variáveis de ambiente
load_dotenv(dotenv_path=PROJECT_ROOT / '.env')

# Configura visualizações
try:
    plt.style.use('seaborn')
except:
    plt.style.use('default')
    
pd.set_option('display.max_columns', None)

print('Configuração concluída!')

In [ ]:
# Carregamento dos dados
from src.data.loaders.market_data_loader import MarketDataLoader

# Define caminho do banco
db_path = PROJECT_ROOT / 'src' / 'data' / 'database' / 'candles.db'
print(f'Tentando acessar banco em: {db_path}')

# Carrega dados
loader = MarketDataLoader(str(db_path))
data = loader.load_data()

print(f'Dados carregados: {len(data)} registros')
print('
Primeiros registros:')
print(data.head())

In [ ]:
# Preparação das features
from src.ml.features.feature_engineering import FeatureEngineering

# Cria features técnicas
engineer = FeatureEngineering()
features = engineer.create_technical_features(data)

# Adiciona features temporais
features = engineer.create_temporal_features(features)

print('Features criadas:')
for col in features.columns:
    print(f'- {col}')

In [ ]:
# Configuração dos agentes
from src.agents.market_agent import RiskManagementAgent, MarketRegimeAgent, FeatureSelectionAgent
from src.agents.agent_coordinator import AgentCoordinator

# Inicializa agentes
risk_agent = RiskManagementAgent(max_drawdown=-0.05)
regime_agent = MarketRegimeAgent(window_size=20)
feature_agent = FeatureSelectionAgent(
    initial_features=features.columns.tolist()
)

# Configura coordenador
coordinator = AgentCoordinator([
    risk_agent,
    regime_agent,
    feature_agent
])

print('Agentes configurados!')